# Stroke Prediction

## Features Information

1) id: unique identifier

2) gender: "Male", "Female" or "Other"

3) age: age of the patient

4) hypertension: 0 if the patient doesn't have hypertension, 1 if the patient has hypertension

5) heart_disease: 0 if the patient doesn't have any heart diseases, 1 if the patient has a heart disease

6) ever_married: "No" or "Yes"

7) work_type: "children", "Govt_jov", "Never_worked", "Private" or "Self-employed"

8) Residence_type: "Rural" or "Urban"

9) avg_glucose_level: average glucose level in blood

10) bmi: body mass index

11) smoking_status: "formerly smoked", "never smoked", "smokes" or "Unknown"*

12) stroke: 1 if the patient had a stroke or 0 if not


*Note: "Unknown" in smoking_status means that the information is unavailable for this patient

Acknowledgements
(Confidential Source) - Use only for educational purposes

## Import necessary libraries

In [1]:
!pip install xgboost


Defaulting to user installation because normal site-packages is not writeable


In [2]:
!pip install imblearn

Defaulting to user installation because normal site-packages is not writeable


In [3]:
!pip install shap

Defaulting to user installation because normal site-packages is not writeable


In [4]:
!pip install optuna

Defaulting to user installation because normal site-packages is not writeable


In [5]:
!pip install catboost

Defaulting to user installation because normal site-packages is not writeable


In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, recall_score, make_scorer
from sklearn.metrics import precision_score, f1_score
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
import shap
import optuna
from sklearn.feature_selection import SelectFromModel
import time

# Try to import CatBoostClassifier
try:
    from catboost import CatBoostClassifier
except ImportError:
    print("CatBoostClassifier is not installed. Skipping CatBoost optimization.")
    CatBoostClassifier = None

In [7]:
# Load and preprocess data
df = pd.read_csv('/Users/Zeryan/Documents/5 Github/stroke EDA-Prediction/healthcare-dataset-stroke-data.csv')

In [8]:
# Encode categorical variables
df_encoded = pd.get_dummies(df, columns=['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status'])

In [9]:
# Impute missing BMI values using a more sophisticated method (e.g., KNN imputer)
from sklearn.impute import KNNImputer


In [10]:
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=5)
df_encoded['bmi'] = imputer.fit_transform(df_encoded[['bmi']])

In [11]:
# Feature engineering
df_encoded['age_squared'] = df_encoded['age'] ** 2
df_encoded['bmi_category'] = pd.cut(df_encoded['bmi'], bins=[0, 18.5, 25, 30, 100], labels=['Underweight', 'Normal', 'Overweight', 'Obese'])
df_encoded = pd.get_dummies(df_encoded, columns=['bmi_category'])

In [29]:
df_encoded.head()

,id,age,hypertension,heart_disease,avg_glucose_level,bmi,stroke,gender_Female,gender_Male,gender_Other,...,Residence_type_Urban,smoking_status_Unknown,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes,age_squared,bmi_category_Underweight,bmi_category_Normal,bmi_category_Overweight,bmi_category_Obese
0,9046,67.0,0,1,228.69,36.600000,1,0,1,0,...,1,0,1,0,0,4489.0,0,0,0,1
1,51676,61.0,0,0,202.21,28.893237,1,1,0,0,...,0,0,0,1,0,3721.0,0,0,1,0
2,31112,80.0,0,1,105.92,32.500000,1,0,1,0,...,0,0,0,1,0,6400.0,0,0,0,1
3,60182,49.0,0,0,171.23,34.400000,1,1,0,0,...,1,0,0,0,1,2401.0,0,0,0,1
4,1665,79.0,1,0,174.12,24.000000,1,1,0,0,...,0,0,0,1,0,6241.0,0,1,0,0


In [12]:
# Split features and target
X = df_encoded.drop(['id', 'stroke'], axis=1)
y = df_encoded['stroke']

In [13]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


In [14]:
# Apply SMOTE to handle class imbalance

smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)



In [15]:
# Define preprocessing steps
numeric_features = ['age', 'avg_glucose_level', 'bmi', 'age_squared']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features)
    ])

In [21]:
# Define models
models = {
    'Random Forest': RandomForestClassifier(random_state=42),
    'XGBoost': XGBClassifier(random_state=42),
    'ExtraTrees': ExtraTreesClassifier(random_state=42),
    'AdaBoost': AdaBoostClassifier(random_state=42),
    'Hist.Grad.Boost': HistGradientBoostingClassifier(random_state=42)
}

# Add CatBoost to models if available
if CatBoostClassifier is not None:
    models['CatBoost'] = CatBoostClassifier(random_state=42)

In [22]:
# Define custom scoring metric (weighted average of recall and ROC AUC)
def custom_scorer(estimator, X, y):
    y_pred = estimator.predict(X)
    recall = recall_score(y, y_pred)
    precision = precision_score(y, y_pred)
    f1 = f1_score(y, y_pred)
    return (recall * 0.4) + (precision * 0.3) + (f1 * 0.3)

In [23]:
# Create a scorer object
custom_scorer_obj = make_scorer(custom_scorer)

In [30]:
print(custom_scorer_obj)

make_scorer(custom_scorer, response_method='predict')


In [28]:
import time

# Hyperparameter tuning using Optuna
def objective(trial, model_class, X, y):
    if model_class == RandomForestClassifier:
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
            'max_depth': trial.suggest_int('max_depth', 3, 20),
            'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
            'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
            'max_features': trial.suggest_float('max_features', 0.1, 1.0),
        }
    elif model_class == XGBClassifier:
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
            'max_depth': trial.suggest_int('max_depth', 3, 20),
            'learning_rate': trial.suggest_loguniform('learning_rate', 1e-3, 1.0),
            'subsample': trial.suggest_uniform('subsample', 0.6, 1.0),
            'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.6, 1.0),
            'gamma': trial.suggest_loguniform('gamma', 1e-8, 1.0),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        }
    elif model_class == CatBoostClassifier:
        params = {
            'iterations': trial.suggest_int('iterations', 100, 1000),
            'depth': trial.suggest_int('depth', 3, 10),
            'learning_rate': trial.suggest_loguniform('learning_rate', 1e-3, 1.0),
            'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-8, 10.0),
            'border_count': trial.suggest_int('border_count', 32, 255),
            'subsample': trial.suggest_uniform('subsample', 0.6, 1.0),
            'colsample_bylevel': trial.suggest_uniform('colsample_bylevel', 0.6, 1.0),
            'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 100),
            'random_strength': trial.suggest_loguniform('random_strength', 1e-8, 10.0),
        }
    elif model_class == ExtraTreesClassifier:
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
            'max_depth': trial.suggest_int('max_depth', 3, 20),
            'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
            'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
            'max_features': trial.suggest_float('max_features', 0.1, 1.0),
            'bootstrap': trial.suggest_categorical('bootstrap', [True, False]),
        }
    elif model_class == AdaBoostClassifier:
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 50, 500),
            'learning_rate': trial.suggest_loguniform('learning_rate', 1e-3, 1.0),
            'algorithm': trial.suggest_categorical('algorithm', ['SAMME', 'SAMME.R']),
        }
    elif model_class == HistGradientBoostingClassifier:
        params = {
            'max_iter': trial.suggest_int('max_iter', 100, 1000),
            'max_depth': trial.suggest_int('max_depth', 3, 20),
            'learning_rate': trial.suggest_loguniform('learning_rate', 1e-3, 1.0),
            'max_leaf_nodes': trial.suggest_int('max_leaf_nodes', 20, 100),
            'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 20),
            'l2_regularization': trial.suggest_loguniform('l2_regularization', 1e-10, 1.0),
            'max_bins': trial.suggest_int('max_bins', 32, 255),
        }

    model = model_class(**params)
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', model)
    ])
    
    scores = cross_val_score(pipeline, X, y, cv=StratifiedKFold(n_splits=5), scoring=custom_scorer_obj)
    return scores.mean()

results = {}

for name, model in models.items():
    print(f"\nOptimizing {name}...")
    study = optuna.create_study(direction='maximize')
    start_time = time.time()
    
    def print_callback(study, trial):
        elapsed_time = time.time() - start_time
        if trial.value is None:
            print(f"Trial {trial.number}: Not yet completed, "
                  f"Best value: {study.best_value:.4f}, "
                  f"Elapsed time: {elapsed_time:.2f} seconds")
        else:
            print(f"Trial {trial.number}: Value: {trial.value:.4f}, "
                  f"Best value: {study.best_value:.4f}, "
                  f"Elapsed time: {elapsed_time:.2f} seconds")
            
    def print_callback(study, trial):
        elapsed_time = time.time() - start_time
        best_value = study.best_value if study.best_trial else "No trials completed"
        if trial.value is None:
            print(f"Trial {trial.number}: Not yet completed, "
                  f"Best value: {best_value}, "
                  f"Elapsed time: {elapsed_time:.2f} seconds")
        else:
            print(f"Trial {trial.number}: Value: {trial.value:.4f}, "
                  f"Best value: {best_value}, "
                  f"Elapsed time: {elapsed_time:.2f} seconds")
    
    study.optimize(lambda trial: objective(trial, type(model), X_train_resampled, y_train_resampled), 
                   n_trials=50, callbacks=[print_callback])
    
    best_params = study.best_params
    best_model = type(model)(**best_params)
    
    print(f"\nBest parameters for {name}:")
    for param, value in best_params.items():
        print(f"  {param}: {value}")
    
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('feature_selection', SelectFromModel(XGBClassifier(random_state=42))),
        ('classifier', best_model)
    ])
    
    print(f"\nFitting {name} pipeline...")
    pipeline.fit(X_train_resampled, y_train_resampled)
    
    print(f"Predicting with {name}...")
    y_pred = pipeline.predict(X_test)
    y_pred_proba = pipeline.predict_proba(X_test)
    
    results[name] = {
        'accuracy': pipeline.score(X_test, y_test),
        'roc_auc': roc_auc_score(y_test, y_pred_proba[:, 1]),
        'recall': recall_score(y_test, y_pred),
        'confusion_matrix': confusion_matrix(y_test, y_pred),
        'classification_report': classification_report(y_test, y_pred),
        'best_params': best_params,
        'pipeline': pipeline
    }
    
    print(f"\nResults for {name}:")
    print(f"  Accuracy: {results[name]['accuracy']:.4f}")
    print(f"  ROC AUC: {results[name]['roc_auc']:.4f}")
    print(f"  Recall: {results[name]['recall']:.4f}")
    print("\nConfusion Matrix:")
    print(results[name]['confusion_matrix'])
    print("\nClassification Report:")
    print(results[name]['classification_report'])
    
    print(f"\nFinished optimizing and evaluating {name}")
    print("=" * 50)

print("\nAll models have been optimized and evaluated.")


[I 2024-07-10 03:25:52,612] A new study created in memory with name: no-name-69a69137-6f98-4cea-9eb3-33c782445566



Optimizing Random Forest...


/Users/Zeryan/Library/Python/3.9/lib/python/site-packages/sklearn/model_selection/_validation.py:1011: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/Zeryan/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_scorer.py", line 137, in __call__
    score = scorer._score(
  File "/Users/Zeryan/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_scorer.py", line 350, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
TypeError: custom_scorer() missing 1 required positional argument: 'y'

  warnings.warn(
/Users/Zeryan/Library/Python/3.9/lib/python/site-packages/sklearn/model_selection/_validation.py:1011: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/Zeryan/Library/Python/3.9/lib/python/site-packag

ValueError: No trials are completed yet.

In [ ]:
# Print results and select the best model
for name, result in results.items():
    print(f"\n{name} Results:")
    print(f"Accuracy: {result['accuracy']:.4f}")
    print(f"ROC AUC: {result['roc_auc']:.4f}")
    print(f"Recall: {result['recall']:.4f}")
    print("Confusion Matrix:")
    print(result['confusion_matrix'])
    print("Classification Report:")
    print(result['classification_report'])
    print("Best Parameters:")
    print(result['best_params'])

best_model = max(results, key=lambda x: results[x]['recall'])
final_model = results[best_model]['pipeline']
final_predictions = final_model.predict(X_test)
final_proba = final_model.predict_proba(X_test)[:, 1]

print("\nFinal Model Evaluation:")
print(f"Model: {best_model}")
print(f"Accuracy: {final_model.score(X_test, y_test):.4f}")
print(f"ROC AUC: {roc_auc_score(y_test, final_proba):.4f}")
print(f"Recall: {recall_score(y_test, final_predictions):.4f}")
print("Confusion Matrix:")
print(confusion_matrix(y_test, final_predictions))
print("Classification Report:")
print(classification_report(y_test, final_predictions))

In [ ]:
# Plot ROC curve
from sklearn.metrics import roc_curve

fpr, tpr, _ = roc_curve(y_test, final_proba)
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f'ROC Curve (AUC = {roc_auc_score(y_test, final_proba):.2f})')
plt.plot([0, 1], [0, 1], linestyle='--', label='Random Classifier')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend()
plt.show()

In [ ]:
# Feature importance analysis using SHAP
explainer = shap.TreeExplainer(final_model.named_steps['classifier'])
shap_values = explainer.shap_values(final_model.named_steps['preprocessor'].transform(X_test))

plt.figure(figsize=(10, 8))
shap.summary_plot(shap_values, X_test, plot_type="bar")
plt.title("Feature Importance (SHAP Values)")
plt.show()

In [ ]:
# Learning curve analysis
from sklearn.model_selection import learning_curve

train_sizes, train_scores, test_scores = learning_curve(
    final_model, X, y, cv=5, n_jobs=-1, 
    train_sizes=np.linspace(0.1, 1.0, 10),
    scoring=make_scorer(custom_scorer)
)

plt.figure(figsize=(10, 6))
plt.plot(train_sizes, np.mean(train_scores, axis=1), label='Training score')
plt.plot(train_sizes, np.mean(test_scores, axis=1), label='Cross-validation score')
plt.xlabel('Training Set Size')
plt.ylabel('Custom Score')
plt.title('Learning Curve')
plt.legend()
plt.show()


## 1. Data Loading and Initial Exploration

In [ ]:
df = pd.read_csv('/kaggle/input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv')

In [ ]:
df.head()

In [ ]:
df.info()

Body

In [ ]:
df.describe().T

## 2. Data Visualization

In [ ]:
numeric_columns = df.select_dtypes(include=[np.number]).columns
print("Numeric columns:", numeric_columns)

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(df[numeric_columns].corr(), annot=True, cmap='coolwarm')
plt.title('Correlation Heatmap (Numeric Features)')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(x='gender', hue='stroke', data=df)
plt.title('Stroke Occurrence by Gender')
plt.show()

In [ ]:
# Encode categorical variables
df_encoded = pd.get_dummies(df, columns=['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status'])

# Create correlation heatmap with encoded features
plt.figure(figsize=(20, 16))
sns.heatmap(df_encoded.corr(), annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Heatmap (All Features Encoded)')
plt.show()

In [ ]:
# Check for missing values
print(df.isnull().sum())

## Handling with missing values

We are going to use rest of data to make a linear regression model to predict missing values of bmi

Let´s work in a copy of original database

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

# Assuming your DataFrame is named 'df'
# Let's create a copy to work with
df_imputed = df_encoded.copy()

Let´s identify the features to use for prediction and prepare the data:

In [ ]:
# Identify numeric and categorical columns
numeric_columns = df_imputed.select_dtypes(include=[np.number]).columns.tolist()
categorical_columns = df_imputed.select_dtypes(include=['object']).columns.tolist()

# Remove 'bmi' from numeric columns if it's there
if 'bmi' in numeric_columns:
    numeric_columns.remove('bmi')

# One-hot encode categorical variables
df_encoded = pd.get_dummies(df_imputed, columns=categorical_columns)

# Prepare features (X) and target (y)
X = df_encoded.drop('bmi', axis=1)
y = df_encoded['bmi']

# Split data into rows with BMI and without BMI
X_with_bmi = X[y.notnull()]
y_with_bmi = y[y.notnull()]
X_without_bmi = X[y.isnull()]

Let´s train the linear regression model

In [ ]:
# Split the data with BMI into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_with_bmi, y_with_bmi, test_size=0.2, random_state=42)

# Initialize and train the model
model = LinearRegression()
model.fit(X_train, y_train)

# Evaluate the model
score = model.score(X_test, y_test)
print(f"R-squared score: {score:.4f}")

Let´s use the model to predict missing BMI values:

In [ ]:
# Predict BMI for rows with missing values
bmi_predicted = model.predict(X_without_bmi)

# Fill in the predicted values
df_imputed.loc[y.isnull(), 'bmi'] = bmi_predicted

# Verify that there are no more missing values in the 'bmi' column
print(df_imputed['bmi'].isnull().sum())

Let´s plot the distribution of original vs imputed BMI values, to check if the imputation looks reasonable:

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(12, 6))
sns.histplot(df['bmi'].dropna(), kde=True, label='Original BMI')
sns.histplot(df_imputed['bmi'], kde=True, label='Imputed BMI')
plt.legend()
plt.title('Distribution of Original vs Imputed BMI')
plt.show()


Our missing values filling method works perfectly.

Let´s continue with Stroke Prediction:

In [ ]:
# Updated correlation heatmap with encoded features
plt.figure(figsize=(20, 16))
sns.heatmap(df_encoded.corr(), annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Heatmap (All Features Encoded)')
plt.show()

In [ ]:
df = df_imputed

In [ ]:
# Split features and target
X = df.drop(['id', 'stroke'], axis=1)
y = df['stroke']


In [ ]:
X.head()

In [ ]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## 4. Handling Class Imbalance

SMOTE (Synthetic Minority Over-sampling Technique) Implementation

In [ ]:
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

## 5. Feature Scaling and Preprocessing Pipeline

In [ ]:
numeric_features = ['age', 'avg_glucose_level', 'bmi']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features)
    ])

## 6. Model Selection and Training

In [ ]:
models = {
    'Random Forest': RandomForestClassifier(random_state=42),
    'XGBoost': XGBClassifier(random_state=42),
    'LightGBM': LGBMClassifier(random_state=42)
}

In [ ]:
results = {}

for name, model in models.items():
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', model)
    ])
    
    pipeline.fit(X_train_resampled, y_train_resampled)
    y_pred = pipeline.predict(X_test)
    
    results[name] = {
        'accuracy': pipeline.score(X_test, y_test),
        'roc_auc': roc_auc_score(y_test, pipeline.predict_proba(X_test)[:, 1]),
        'confusion_matrix': confusion_matrix(y_test, y_pred),
        'classification_report': classification_report(y_test, y_pred)
    }

In [ ]:
for name, result in results.items():
    print(f"\n{name} Results:")
    print(f"Accuracy: {result['accuracy']:.4f}")
    print(f"ROC AUC: {result['roc_auc']:.4f}")
    print("Confusion Matrix:")
    print(result['confusion_matrix'])
    print("Classification Report:")
    print(result['classification_report'])

### 8. Feature Importance Analysis

In [ ]:
best_model = max(results, key=lambda x: results[x]['roc_auc'])
best_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', models[best_model])
])
best_pipeline.fit(X_train_resampled, y_train_resampled)

## 10. Final Model Evaluation

In [ ]:
final_model = best_pipeline
final_predictions = final_model.predict(X_test)
final_proba = final_model.predict_proba(X_test)[:, 1]

print("\nFinal Model Evaluation:")
print(f"Accuracy: {final_model.score(X_test, y_test):.4f}")
print(f"ROC AUC: {roc_auc_score(y_test, final_proba):.4f}")
print("Confusion Matrix:")
print(confusion_matrix(y_test, final_predictions))
print("Classification Report:")
print(classification_report(y_test, final_predictions))

## 11. ROC Curve Visualization

In [ ]:

from sklearn.metrics import roc_curve

fpr, tpr, _ = roc_curve(y_test, final_proba)
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f'ROC Curve (AUC = {roc_auc_score(y_test, final_proba):.2f})')
plt.plot([0, 1], [0, 1], linestyle='--', label='Random Classifier')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend()
plt.show()